## Introduction to the TAP service and database queries
This notebook demos how to access the TAP service that sits in front of the qserv database and gives examples of some simple queries.

Additional information the infrared fusion database is given in the confluence doc:  
<a href="https://lsst-uk.atlassian.net/wiki/spaces/LUSC/pages/3292463109/Working+with+the+LSST-VISTA+Fusion+database">Working with the LSST-VISTA Fusion database</a>

Import modules and checj the TAP service

In [2]:
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

service = get_tap_service("tap")
assert service is not None
assert service.baseurl == "https://rsp.lsst.ac.uk/api/tap"

print(service.maxrec)
print(service.hardlimit)

134217728
134217728


See what databases the TAP service can access, and get resultset as astropy table


In [3]:
query = "SELECT * FROM tap_schema.schemas"
resultsSchema = service.search(query).to_table()
resultsSchema

description,schema_index,schema_name,utype
str512,int32,str64,str512
no database description provided,1,dc2_run2_1i_dr1b,
gaia_source catwise_2020 matches from Edinburgh,2,gaiaxcatwise,
allsky gaia_source catwise_2020 matches from Edinburgh,3,gaiaxcatwiseallsky,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
no database description provided,0,UDS,
UWS Metadata,120000,uws,
VIDEO/HSC database from WP3.5,4,video,
VIKING/HSC database from WP3.5,5,viking,


see what tables are in the VIDEO database

In [4]:
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = '" \
        + 'video' + "' order by table_index ASC"
print(query)

results = service.search(query)
results = results.to_table()
results

SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = 'video' order by table_index ASC


description,schema_name,table_index,table_name,table_type,utype
str512,str512,int32,str64,str8,str512
merged table,video,0,video.merged,table,
hsc_g_f measured results,video,0,video.hsc_g_f,table,
hsc_g_m measured results,video,0,video.hsc_g_m,table,
hsc_i_f measured results,video,0,video.hsc_i_f,table,
hsc_i_m measured results,video,0,video.hsc_i_m,table,
hsc_r_f measured results,video,0,video.hsc_r_f,table,
hsc_r_m measured results,video,0,video.hsc_r_m,table,
hsc_y_f measured results,video,0,video.hsc_y_f,table,
hsc_y_m measured results,video,0,video.hsc_y_m,table,



run a cone query of the <b>merged</b> table selecting the various mags


In [11]:
query = """SELECT hsc_r_m_coord_ra as ra, hsc_r_m_coord_dec as dec,
       hsc_g_m_base_circularapertureflux_6_0_mag as g,
       hsc_r_m_base_circularapertureflux_6_0_mag as r,hsc_i_m_base_circularapertureflux_6_0_mag as i,
       hsc_z_m_base_circularapertureflux_6_0_mag as z,hsc_y_m_base_circularapertureflux_6_0_mag as y,
       vircam_z_m_base_circularapertureflux_6_0_mag as vZ,vircam_y_m_base_circularapertureflux_6_0_mag as vY,
       vircam_j_m_base_circularapertureflux_6_0_mag as J, vircam_h_m_base_circularapertureflux_6_0_mag as H,
       vircam_ks_m_base_circularapertureflux_6_0_mag as K 
FROM video.merged 
WHERE CONTAINS(POINT('ICRS', hsc_r_m_coord_ra, hsc_r_m_coord_dec),CIRCLE('ICRS', 36, -5, 0.016666666666666666))=1"""
results = service.search(query)
results.to_table()

ra,dec,g,r,i,z,y,vZ,vY,J,H,K
float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
35.997668055572646,-5.012762096521284,21.1462,20.0168,19.5063,19.2597,19.0787,19.4497,19.1858,18.8189,18.4942,18.206
36.00644905183114,-5.014298879131164,23.5438,22.7717,22.4788,22.3809,22.1564,22.5476,22.2646,22.2332,21.9897,21.89
35.99621147124081,-5.013280452168841,25.3675,25.0273,24.5536,24.2729,24.1416,24.4387,24.4648,24.2061,24.2031,23.6728
35.99542208939584,-5.015639959094359,26.0834,25.5603,24.8588,24.603,24.3392,24.6028,24.3497,24.1385,23.8535,24.0075
35.99927836140051,-5.013912066268159,26.942,25.6749,24.7923,24.2022,23.4323,24.5154,23.485,22.9364,22.0742,21.5446
36.000272497131206,-5.014486067031779,25.8131,25.4237,24.8091,24.3961,23.9021,24.46,23.9489,23.6811,23.2801,23.1479
35.99580306651626,-5.0128925556492785,25.5509,25.4257,25.2706,25.3032,25.9004,25.7723,25.0809,25.469,24.4855,24.3444
35.996265513127014,-5.01552752482956,27.0539,25.6366,25.3904,25.1492,25.1248,25.2351,25.0514,25.2712,24.5474,24.5885
35.99962357667754,-5.016348483138161,26.7941,25.8118,25.7111,25.7408,25.1196,25.5295,25.9734,26.2866,25.8118,24.6133



Similar query but now joining with one of the other single band tables


In [12]:
query = """SELECT hsc_r_m_coord_ra as ra, hsc_r_m_coord_dec as dec,m.objectId,
       m.hsc_g_m_base_circularapertureflux_6_0_mag as g,
       hsc_r_m_base_circularapertureflux_6_0_mag as r,hsc_i_m_base_circularapertureflux_6_0_mag as i,
       hsc_z_m_base_circularapertureflux_6_0_mag as z,hsc_y_m_base_circularapertureflux_6_0_mag as y,
       vircam_z_m_base_circularapertureflux_6_0_mag as vZ,vircam_y_m_base_circularapertureflux_6_0_mag as vY,
       vircam_j_m_base_circularapertureflux_6_0_mag as J, vircam_h_m_base_circularapertureflux_6_0_mag as H,
       vircam_ks_m_base_circularapertureflux_6_0_mag as K,
       g.hsc_g_m_base_circularapertureflux_6_0_mag as g2
FROM video.merged as m join video.hsc_g_m as g on m.objectId=g.objectId
WHERE CONTAINS(POINT('ICRS', hsc_r_m_coord_ra, hsc_r_m_coord_dec),CIRCLE('ICRS', 36, -5, 0.016666666666666666))=1"""
results = service.search(query)
results.to_table()

ra,dec,objectId,g,r,i,z,y,vZ,vY,J,H,K,g2
float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
35.997668055572646,-5.012762096521284,3036811533498715972,21.1462,20.0168,19.5063,19.2597,19.0787,19.4497,19.1858,18.8189,18.4942,18.206,21.1462
36.00644905183114,-5.014298879131164,3036811533498715942,23.5438,22.7717,22.4788,22.3809,22.1564,22.5476,22.2646,22.2332,21.9897,21.89,23.5438
35.99621147124081,-5.013280452168841,3036811533498737963,25.3675,25.0273,24.5536,24.2729,24.1416,24.4387,24.4648,24.2061,24.2031,23.6728,25.3675
35.99542208939584,-5.015639959094359,3036811533498737964,26.0834,25.5603,24.8588,24.603,24.3392,24.6028,24.3497,24.1385,23.8535,24.0075,26.0834
35.99927836140051,-5.013912066268159,3036811533498737965,26.942,25.6749,24.7923,24.2022,23.4323,24.5154,23.485,22.9364,22.0742,21.5446,26.942
36.000272497131206,-5.014486067031779,3036811533498737966,25.8131,25.4237,24.8091,24.3961,23.9021,24.46,23.9489,23.6811,23.2801,23.1479,25.8131
35.99580306651626,-5.0128925556492785,3036811533498737967,25.5509,25.4257,25.2706,25.3032,25.9004,25.7723,25.0809,25.469,24.4855,24.3444,25.5509
35.996265513127014,-5.01552752482956,3036811533498737968,27.0539,25.6366,25.3904,25.1492,25.1248,25.2351,25.0514,25.2712,24.5474,24.5885,27.0539
35.99962357667754,-5.016348483138161,3036811533498737969,26.7941,25.8118,25.7111,25.7408,25.1196,25.5295,25.9734,26.2866,25.8118,24.6133,26.7941


In [13]:
print(type(results))
table=results.to_table()
print(type(table))

<class 'pyvo.dal.tap.TAPResults'>
<class 'astropy.table.table.Table'>


Below we obtain a sample of objectIds and then use those in a set of queries

In [6]:
query="select objectId from video.merged order by objectId limit 10000"
results = service.search(query)
objIdTable = results.to_table()
print(objIdTable)

      objectId     
-------------------
3036327748382490625
3036327748382490626
3036327748382490627
3036327748382490628
3036327748382490629
3036327748382490630
3036327748382490631
3036327748382490632
3036327748382490633
3036327748382490634
                ...
3036327748382511553
3036327748382511554
3036327748382511555
3036327748382511556
3036327748382511557
3036327748382511558
3036327748382511559
3036327748382511560
3036327748382511561
3036327748382511562
3036327748382511563
Length = 10000 rows


In [7]:
objectIds=objIdTable['objectId'].data
print(objectIds[0],max(objectIds),min(objectIds))
print(len(objectIds))

3036327748382490625 3036327748382511563 3036327748382490625
10000


loop through the objectIds in batches and then submit queries of the form
... where objectId in (.,.,.,)
and pull out the info from another table

In [9]:
import numpy
chunks = numpy.array_split(numpy.array(objectIds),10)
import datetime
i=0
from astropy.table import QTable, Table, Column, vstack
fullResults=Table()
for chunk in chunks:
    print(i,len(chunk),datetime.datetime.now())
    
    inClause=','.join(str(objId) for objId in chunk) #numpy.array2string(chunk, separator=",")
    #print(inClause)
    query="select * from video.vir_k_f where objectId in ("+inClause+")"
    #print(query)
    results = service.search(query)
    table = results.to_table()
    if i==0:
        fullResults=table
    else:
        fullResults=(vstack([fullResults, table]))
    #print(table)
    i+=1
print(len(fullResults))

0 1000 2023-11-20 14:00:59.564880
1 1000 2023-11-20 14:01:44.514594
2 1000 2023-11-20 14:02:09.590983
3 1000 2023-11-20 14:02:45.059237
4 1000 2023-11-20 14:03:35.725699
5 1000 2023-11-20 14:03:50.721035
6 1000 2023-11-20 14:04:15.670907
7 1000 2023-11-20 14:05:10.694693
8 1000 2023-11-20 14:05:41.335129
9 1000 2023-11-20 14:05:56.907277
10000


In [10]:
print(fullResults)

chunkId       objectId      ... vir_k_f_slot_shape_xy vir_k_f_slot_shape_yy
------- ------------------- ... --------------------- ---------------------
  30274 3036327748382490625 ...              0.534636               9.10167
  30274 3036327748382490626 ...             -0.713247               5.47227
  30274 3036327748382490627 ...             -0.147149                2.2759
  30274 3036327748382490628 ...             -0.516365               1.96049
  30274 3036327748382490629 ...              0.170879               3.35284
  30274 3036327748382490630 ...               3.24481               7.86276
  30274 3036327748382490631 ...               0.13163               3.79302
  30274 3036327748382490632 ...              0.139035               3.84471
  30274 3036327748382490633 ...                  1.89               5.68733
  30274 3036327748382490634 ...              -1.91697                4.9075
    ...                 ... ...                   ...                   ...
  30274 3036

cleanup ?

In [11]:
fullResults=None
del fullResults
import gc
gc.collect()

5700